

---


**更新紀錄：**
*   114.04.05 修正專案說明
*   113.11.26 順應網站改版 修正更新程式碼
*   110.12.26 更新colab專用driver版


---
**專案名稱：** 找個實習吧

**專案目的：**
希望透過網路爬蟲，簡化找尋實習時搜索的步驟，藉此將不同人力銀行職缺做歸類整併為自定義的16類，最後彙整出dataframe/ excel表格後供後續使用。

**說明：**
完整專案為三大人力銀行爬蟲，此頁面僅展示當中本人負責之子專案（104人力銀行）爬蟲運行部分，也因應展示需求只程式只會運行『3種』職缺類別爬蟲。

**關鍵字：** 網路爬蟲、python、selenium

**完整專案demo影片：**
https://www.youtube.com/watch?v=qBZu3-dxNCU



In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install pyquery

import pandas as pd
import requests, time
from selenium import webdriver
from pyquery import PyQuery as pq


# 用python建立一個Excel空白活頁簿
#excel_file = Workbook()
# 建立一個工作中表
#sheet = excel_file.active



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 19.6 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,978 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubun

In [ ]:
# 先填入第一列的欄位名稱(110.12.26修改為存成存成list) (113.11.26 因應網站改版 程式碼更新)
colname=['職缺名稱','公司','縣市','地區','職缺種類','職缺連結']
datalist = []

###根據預設好的的標籤分類
summaryDict={"1":"人資／經營",
             "2":"行政╱總務╱法務",
             "3":"行銷╱企劃╱專案管理",
             "4":"金融保險／財會／稽核",
             "5":"客服╱門市╱業務╱貿易",
             "6":"餐飲╱旅遊 ╱美容美髮",
             "7":"資訊軟體系統類",
             "8":"操作╱技術╱維修",
             "9":"資材╱物流╱運輸",
             "10":"營建╱製圖類／測量",
             "11":"傳播╱文字╱設計",
             "12":"醫療／護理／保健",
             "13":"學術╱教育╱輔導",
             "14":"製造／品管／環衛",
             "15":"軍警消╱保全類",
             "16":"其他職務"}

###根據標籤所需要的網址指引
jobcatDict=["2001000000",
            "2002000000",
            "2004000000",
            "2009000000",
            "2005000000",
            "2006000000",
            "2007000000",
            "2010000000",
            "2011000000",
            "2012000000",
            "2014000000%2C2013000000",
            "2015000000",
            "2016000000",
            "2009000000",
            "2017000000",
            "2008000000%2C2018000000"]

In [ ]:
###儲存爬蟲data
totalJob=[]
totalCon=[]
totalArea=[]
totalArea2=[]
totalurlList=[]

###要爬蟲所預設的條件
inputData = {'ro':'0', 'keyword':'暑期實習', 'isnew':'30', 'mode':'l','jobcat':"0"}

###110.12.26新增解決chromedriver問題
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

###在這邊第一個回圈是讓我的inputData改變，來做重複16次的動作（根據要爬的網址不同）
for i in range(3):
    job=[]
    con=[]
    area=[]
    area2=[]
    urlList=[]


    inputData['jobcat']= jobcatDict[i]
    url = requests.get("https://www.104.com.tw/jobs/search/?", inputData, allow_redirects=False).url
    #print(url)
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    ##本機有載好chromedriver才適用的code
    #url = requests.get("https://www.104.com.tw/jobs/search/?", inputData, allow_redirects=False).url
    #driver = webdriver.Chrome(executable_path="/Users/Ponpon/Downloads/chromedriver")
    #driver.get(url)

    ###因為是動態網頁（固定將頁面捲到頁底）
    for j in range(10):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(0.5)


    ###移除掉廣告雜訊的data 113.11.25 網頁更改移除
    pqSource = pq(driver.page_source)
    pqSource.remove(".b-block--ad")


    ########工作名稱的解析
    jobvar=0
    for k in pqSource('.info-name').items():
        jobvar += 1
        #print("{} {}".format(jobvar,k.text()))
        #job_title = k.text()
        #print(job_title)
        job.append(k.text())       ###將工作名稱加到job的dataList裡
        totalJob.append(k.text())



    ########公司名稱的解析
    companyVar=0
    for l in pqSource('.info-company').items():
        companyVar += 1
        #print("{} {}".format(companyVar,l.text()))
        #print(l("a.jb-link-blue").text())
        con.append(l("a.jb-link-blue").text())       ###將公司名稱加到con的dataList裡
        totalCon.append(l("a.jb-link-blue").text())



    ########職位縣市行政區的解析
    areaVar=0
    count=0
    for m in pqSource('.info-tags .info-tags__text').items():
            count += 1
            #print(count)
            if count%4==1:
              #print(m.text())

              areaVar += 1
              if m.text()[0:3] == "新竹縣":
                  #print("I am here")
                  area.append("新竹縣市")
                  totalArea.append("新竹縣市")

              elif m.text()[0:3] == "新竹市":
                  #print("I am here")
                  area.append("新竹縣市")
                  totalArea.append("新竹縣市")
              else:
                  area.append(m.text()[0:3])      ###將地區名稱加到area的dataList裡
                  totalArea.append(m.text()[0:3])
                  #print("I am not here") ########地區名稱的解析

              area2.append(m.text()[3:])      ###將地區名稱加到area2的dataList裡
              totalArea2.append(m.text()[3:])
            else:
              continue


    ########行政區的解析
    #area2Var=0
    #for p in pqSource('.job-mode__area').items():
        #if area2Var==0:
            #area2Var += 1
            #continue
        #print("{} {}".format(i,j.text()))
        #area2Var += 1
        #area2.append(p.text()[3:])      ###將地區名稱加到area2的dataList裡
        #totalArea2.append(p.text()[3:])

    ########URL的補充
    for q in pqSource('.info-job').items():
        h=q.attr['href']
        #print(h[2:])
        urlList.append( str(h)  )     ###將工作名稱加到job的dataList裡
        totalurlList.append(str(h))


    # 使用迴圈逐列增加，使用append的方式往下新增資料到excel
    for n in range(len(job)):
        columnA = job[n]
        columnB = con[n]
        columnC = area[n]
        columnD = area2[n]
        columnE = summaryDict[str(i+1)]
        columnF = urlList[n]
        # 實際將資料寫入每一列
        datalist.append([columnA, columnB, columnC, columnD, columnE, columnF])
    print("已處理完{}/16筆資料".format(i+1))
    print(len(job))
## 儲存成XLSX檔
#excel_file.save('工作職缺.xlsx')


print("總共有：{}筆職缺已新增完成".format(len(totalJob)))

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file
已處理完1/16筆資料
81
已處理完2/16筆資料
90
已處理完3/16筆資料
120
總共有：291筆職缺已新增完成


In [ ]:
df=pd.DataFrame(datalist,columns=colname)
df.index+=1
df.head(20)

,職缺名稱,公司,縣市,地區,職缺種類,職缺連結
1,[2025 暑期實習計畫－優先獲得畢業預聘機會 ] 人力資源部實習 Human Resour...,德州儀器工業股份有限公司,台北市,松山區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
2,【花旗銀行】2025暑期實習&儲備主管計畫 (Citi Summer & Full Time...,花旗(台灣) 商業銀行股份有限公司,台北市,信義區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
3,客戶成功 & 人力資源實習生 (BD & HR Internship),威德數位設計有限公司,台中市,北區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
4,REQ_24087031 AA0220-教育訓練實習生,啟碁科技股份有限公司,新竹縣市,,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
5,營運實習生 Operations Intern,加密實驗股份有限公司,台北市,中正區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
6,Shopee APP - 校園活動實習生Campus Recruitment Intern ...,新加坡商蝦皮娛樂電商有限公司台灣分公司,台北市,信義區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
7,實習生-人事行政助理 (非寒暑假短期 & 歡迎應屆畢業生）,玩美行動股份有限公司,台北市,內湖區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
8,【實習】南寶樹脂菁英實習,南寶樹脂化學工廠股份有限公司,台南市,西港區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
9,實習生(商管領域),群創管理顧問有限公司,台北市,中正區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
10,[學生專區][兼職首選] Intern 實習生 (提供轉正職機會) - 台北&新竹,美商佰德管理顧問有限公司台灣分公司,台北市,內湖區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...


In [ ]:
######用來debug檢查工作用

k=0
for i in range(len(job)):
    k+=1
    print("data"+str(k))
    print("Job:{}".format(job[i]))
    print("Company:{}".format(con[i]))
    print("Area:{}".format(area[i]))
    print("")

In [ ]:
#######讀取資料
#df = pd.read_excel("Jobdata.xlsx")
#df.head(15)
#df[df.Area == "台北市松山區"]

df.to_excel("工作職缺.xlsx")
df[df.縣市 == "台北市"]

,職缺名稱,公司,縣市,地區,職缺種類,職缺連結
1,[2025 暑期實習計畫－優先獲得畢業預聘機會 ] 人力資源部實習 Human Resour...,德州儀器工業股份有限公司,台北市,松山區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
2,【花旗銀行】2025暑期實習&儲備主管計畫 (Citi Summer & Full Time...,花旗(台灣) 商業銀行股份有限公司,台北市,信義區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
5,營運實習生 Operations Intern,加密實驗股份有限公司,台北市,中正區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
6,Shopee APP - 校園活動實習生Campus Recruitment Intern ...,新加坡商蝦皮娛樂電商有限公司台灣分公司,台北市,信義區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
7,實習生-人事行政助理 (非寒暑假短期 & 歡迎應屆畢業生）,玩美行動股份有限公司,台北市,內湖區,人資／經營,https://104app.onelink.me/zB8j/redirect?af_r=h...
...,...,...,...,...,...,...
256,實習生-菲律賓市場行銷助理(非短期寒暑假實習),玩美行動股份有限公司,台北市,內湖區,行銷╱企劃╱專案管理,https://104app.onelink.me/zB8j/redirect?af_r=h...
258,【ESG策略與永續發展諮詢服務】永續實習生,安永聯合會計師事務所,台北市,信義區,行銷╱企劃╱專案管理,https://104app.onelink.me/zB8j/redirect?af_r=h...
259,【Taiwan Mobile 實習計畫】Intern-品牌活動與社群經營,台灣大哥大股份有限公司,台北市,信義區,行銷╱企劃╱專案管理,https://104app.onelink.me/zB8j/redirect?af_r=h...
261,【Taiwan Mobile 實習計畫】Intern-電信金融行銷設計,台灣大哥大股份有限公司,台北市,信義區,行銷╱企劃╱專案管理,https://104app.onelink.me/zB8j/redirect?af_r=h...
